In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score

# Set the paths for train and validation directories
train_dir = '../../../../senne/OneDrive/Bureaublad/datasets/RocketVsCarrots/Train'
validation_dir = '../../../../senne/OneDrive/Bureaublad/datasets/RocketVsCarrots/Validation'

# Define the batch size
batch_size = 32

# Define the image size
img_size = (224, 224)

# Define the number of classes
num_classes = 2

# Data augmentation for train set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data augmentation for validation set (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the train set
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

# Load and preprocess the validation set
validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                       target_size=img_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size,
          epochs=10)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Make predictions on the validation set
val_predictions = model.predict(validation_generator)
val_pred_labels = tf.argmax(val_predictions, axis=1).numpy()

# Get true labels of the validation set
val_true_labels = validation_generator.classes

# Calculate accuracy score
accuracy = accuracy_score(val_true_labels, val_pred_labels)
print(f"Accuracy Score: {accuracy}")


AttributeError: module 'collections' has no attribute 'MutableMapping'

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.metrics import accuracy_score
import pathlib

# Set the path for the validation directory
validation_dir = '../../../../senne/OneDrive/Bureaublad/datasets/RocketVsCarrots/Validation'

# Define the batch size
batch_size = 32

# Define the image size
img_size = (224, 224)

# Define the number of classes
num_classes = 2

# Data augmentation for validation set (only rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the validation set
validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                       target_size=img_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Flatten the output of the base model
flatten = tf.keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with softmax activation
output = tf.keras.layers.Dense(num_classes, activation='softmax')(flatten)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Load the weights of the pre-trained model
model.load_weights('pretrained_model_weights.h5')

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Make predictions on the validation set
val_predictions = model.predict(validation_generator)
val_pred_labels = tf.argmax(val_predictions, axis=1).numpy()

# Get true labels of the validation set
val_true_labels = validation_generator.classes

# Calculate accuracy score
accuracy = accuracy_score(val_true_labels, val_pred_labels)
print(f"Accuracy Score: {accuracy}")


AttributeError: module 'collections' has no attribute 'MutableMapping'